In [ ]:
# 1. 필수 패키지 설치
!pip install -U konlpy openpyxl scikit-learn keybert transformers
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'




  Using cached konlpy-0.6.0-py2.py3-none-any.whl.metadata (1.9 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 M

In [ ]:
# ✅ 2. 전처리된 CSV 업로드
from google.colab import files
uploaded = files.upload()  #전처리는 특수기호 제거만 적용

# ✅ 3. 데이터 불러오기
import pandas as pd
import torch
from transformers import BertModel
from kobert_tokenizer import KoBERTTokenizer
from keybert import KeyBERT
from tqdm import tqdm

df = pd.read_csv("리빙도서_상품명_전처리_통합.csv")

# ✅ 4. KoBERT 임베딩 모델 정의
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertModel.from_pretrained('skt/kobert-base-v1')
model.eval()

class KoBERTEmbedder:
    def embed(self, documents, **kwargs):
        return [get_cls_embedding(doc) for doc in documents]

def get_cls_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=64, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[0][0].numpy()

kw_model = KeyBERT(model=KoBERTEmbedder())

# ✅ 5. 키워드 추출 함수 정의
def extract_keywords_unique(text, gram_type=(1, 2), top_n=4):
    if not isinstance(text, str) or text.strip() == "":
        return ""
    try:
        keywords = kw_model.extract_keywords(
            text,
            top_n=10,
            keyphrase_ngram_range=gram_type,
            stop_words=None
        )
        unique_keywords = []
        for kw, _ in keywords:
            if not any((kw in existing or existing in kw) for existing in unique_keywords):
                unique_keywords.append(kw)
            if len(unique_keywords) == top_n:
                break
        return ", ".join(unique_keywords)
    except:
        return ""

# ✅ 6. 키워드 추출 실행
df["keywords"] = [extract_keywords_unique(text) for text in tqdm(df["상품명_전처리"].fillna(""))]

# ✅ 7. 상품명_전처리 열 제거 후 저장
df.drop(columns=["상품명_전처리"], inplace=True)
df.to_csv("리빙_통합.csv", index=False)

# ✅ 8. 다운로드
files.download("리빙_통합.csv")

Saving 리빙도서_상품명_전처리_통합.csv to 리빙도서_상품명_전처리_통합 (2).csv


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
100%|██████████| 13000/13000 [07:14<00:00, 29.94it/s]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>